In [5]:
import keras
keras.__version__


'2.3.1'

In [6]:
import numpy as np
import pandas as pd

In [7]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 62 bytes


In [8]:
!mkdir $HOME/.kaggle

In [9]:
!mv kaggle.json $HOME/.kaggle/kaggle.json
!chmod 600 $HOME/.kaggle/kaggle.json

In [10]:
from keras.datasets import cifar10

In [11]:
# load dataset
(trainX, trainy), (testX, testy) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [12]:
# summarize loaded dataset
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape, testy.shape))

Train: X=(50000, 32, 32, 3), y=(50000, 1)
Test: X=(10000, 32, 32, 3), y=(10000, 1)


In [13]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(32, 32, 3),)

In [14]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [15]:
from keras.utils import to_categorical

trainy = to_categorical(trainy)
testy = to_categorical(testy)

In [16]:
# scale pixels

# convert from integers to floats
trainX = trainX.astype('float32')
testX = testX.astype('float32')

# normalize to range 0-1
trainX = trainX / 255.0
testX = testX / 255.0


In [17]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras import models
from keras import layers
from keras import optimizers

In [59]:
model = models.Sequential()

model.add(conv_base)
model.add(Flatten())

model.add(layers.Dense(130, activation='relu'))

model.add(layers.Dropout(0.5))

model.add(layers.Dense(160, activation='relu'))
model.add(layers.Dense(190, activation='relu'))
model.add(layers.Dense(220, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [60]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='categorical_crossentropy',
              metrics=['acc'])

In [61]:
history = model.fit(trainX, trainy,
                    epochs= 5,
                    batch_size=20,
                    validation_data=(testX, testy))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 68s 1ms/step - loss: 0.4101 - acc: 0.8811 - val_loss: 0.5728 - val_acc: 0.8526
Epoch 2/5
50000/50000 [==============================] - 68s 1ms/step - loss: 0.2553 - acc: 0.9255 - val_loss: 0.6060 - val_acc: 0.8511
Epoch 3/5
50000/50000 [==============================] - 69s 1ms/step - loss: 0.2315 - acc: 0.9316 - val_loss: 0.6682 - val_acc: 0.8536
Epoch 4/5
50000/50000 [==============================] - 68s 1ms/step - loss: 0.2171 - acc: 0.9362 - val_loss: 0.6367 - val_acc: 0.8527
Epoch 5/5
50000/50000 [==============================] - 68s 1ms/step - loss: 0.2074 - acc: 0.9409 - val_loss: 0.7026 - val_acc: 0.8563


In [21]:
# make prediction.
predictions = model.predict(testX)

# Creamos un modelo nuevo

In [48]:
from keras.applications import VGG16
conv_base2 = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(32, 32, 3),)

In [49]:
conv_base2.trainable = True

set_trainable = False
for layer in conv_base2.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False 

In [56]:
model2 = models.Sequential()

model2.add(conv_base2)
model2.add(Flatten())

model2.add(layers.Dense(160, activation='relu'))

model2.add(layers.Dropout(0.5))

model2.add(layers.Dense(190, activation='relu'))
model2.add(layers.Dense(220, activation='relu'))

model2.add(layers.Dense(10, activation='softmax'))

In [57]:
model2.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

In [58]:
history2 = model2.fit(trainX, trainy,
                    epochs= 5,
                    batch_size=20,
                    validation_data=(testX, testy))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 67s 1ms/step - loss: 1.1251 - acc: 0.6204 - val_loss: 0.8602 - val_acc: 0.7038
Epoch 2/5
50000/50000 [==============================] - 67s 1ms/step - loss: 0.8632 - acc: 0.7116 - val_loss: 0.8164 - val_acc: 0.7207
Epoch 3/5
50000/50000 [==============================] - 67s 1ms/step - loss: 0.7929 - acc: 0.7356 - val_loss: 0.8003 - val_acc: 0.7266
Epoch 4/5
50000/50000 [==============================] - 67s 1ms/step - loss: 0.7441 - acc: 0.7510 - val_loss: 0.7782 - val_acc: 0.7367
Epoch 5/5
50000/50000 [==============================] - 67s 1ms/step - loss: 0.7041 - acc: 0.7661 - val_loss: 0.7638 - val_acc: 0.7454
